In [51]:
import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats
import math
import re
import subprocess
from os import listdir
from os.path import isfile, join

In [2]:
import matplotlib as mpl 
mpl.use('Agg')
mpl.rcParams['pdf.fonttype'] = 42
import seaborn as sns
import matplotlib.pyplot as plt
plt.close("all")
mpl.rcParams['font.sans-serif'] = "Arial"
mpl.rcParams["font.family"] = 'sans-serif'
mpl.rcParams['font.size'] = 8
sns.set(style="whitegrid")

In [52]:
cd /Users/davidy/jamboree20crispr/ontarget_analysis/230724_treg_analysis

/Users/davidy/jamboree20crispr/ontarget_analysis/230724_treg_analysis


In [ ]:
"""
Notes:
mm10
guideScan output that we used to give each targeting gRNA an accessibility score based on the overlap with Treg ATAC-seq. 



"""

In [53]:
##### Process guide count files
FF_df = pd.read_csv('220304_TG2_GITR_KRAB_Output_CountTable.txt', sep = '\t')

### Compute log2FCs: (H/L)
rep1_hi_counts = FF_df['TG2-GITR-Hi-rep1'].sum()
rep2_hi_counts = FF_df['TG2-GITR-Hi-rep2'].sum()
rep3_hi_counts = FF_df['TG2-GITR-Hi-rep3'].sum()
rep4_hi_counts = FF_df['TG2-GITR-Hi-rep4'].sum()
rep1_low_counts = FF_df['TG2-GITR-Lo-rep1'].sum()
rep2_low_counts = FF_df['TG2-GITR-Lo-rep2'].sum()
rep3_low_counts = FF_df['TG2-GITR-Lo-rep3'].sum()
rep4_low_counts = FF_df['TG2-GITR-Lo-rep4'].sum()
FF_df['log2_rep1'] = np.log2((1+FF_df['TG2-GITR-Hi-rep1']/FF_df['TG2-GITR-Hi-rep1'].mean()) / (1+FF_df['TG2-GITR-Lo-rep1']/FF_df['TG2-GITR-Lo-rep1'].mean()))
FF_df['log2_rep2'] = np.log2((1+FF_df['TG2-GITR-Hi-rep2']/FF_df['TG2-GITR-Hi-rep2'].mean()) / (1+FF_df['TG2-GITR-Lo-rep2']/FF_df['TG2-GITR-Lo-rep2'].mean()))
FF_df['log2_rep3'] = np.log2((1+FF_df['TG2-GITR-Hi-rep3']/FF_df['TG2-GITR-Hi-rep3'].mean()) / (1+FF_df['TG2-GITR-Lo-rep3']/FF_df['TG2-GITR-Lo-rep3'].mean()))
FF_df['log2_rep4'] = np.log2((1+FF_df['TG2-GITR-Hi-rep4']/FF_df['TG2-GITR-Hi-rep4'].mean()) / (1+FF_df['TG2-GITR-Lo-rep4']/FF_df['TG2-GITR-Lo-rep4'].mean()))
#FF_df['avg_log2'] = (FF_df['log2_rep1'] + FF_df['log2_rep2'] + FF_df['log2_rep3'] + FF_df['log2_rep4'])/4

### Z-transform based on distribution of negative controls
rep1_nt_mean = np.mean(FF_df.loc[FF_df['locus']=='nt']['log2_rep1'])
rep1_nt_std = np.std(FF_df.loc[FF_df['locus']=='nt']['log2_rep1'], ddof=1)
rep2_nt_mean = np.mean(FF_df.loc[FF_df['locus']=='nt']['log2_rep2'])
rep2_nt_std = np.std(FF_df.loc[FF_df['locus']=='nt']['log2_rep2'], ddof=1)
rep3_nt_mean = np.mean(FF_df.loc[FF_df['locus']=='nt']['log2_rep3'])
rep3_nt_std = np.std(FF_df.loc[FF_df['locus']=='nt']['log2_rep3'], ddof=1)
rep4_nt_mean = np.mean(FF_df.loc[FF_df['locus']=='nt']['log2_rep4'])
rep4_nt_std = np.std(FF_df.loc[FF_df['locus']=='nt']['log2_rep4'], ddof=1)
#nt_log2fc_avg_mean = np.mean(FF_df.loc[FF_df['locus']=='nt']['avg_log2'])
#nt_log2fc_avg_sd = np.std(FF_df.loc[FF_df['locus']=='nt']['avg_log2'], ddof=1)

FF_df['z_log2_rep1'] = (FF_df['log2_rep1'] - rep1_nt_mean) / rep1_nt_std
FF_df['z_log2_rep2'] = (FF_df['log2_rep2'] - rep2_nt_mean) / rep2_nt_std
FF_df['z_log2_rep3'] = (FF_df['log2_rep3'] - rep3_nt_mean) / rep3_nt_std
FF_df['z_log2_rep4'] = (FF_df['log2_rep4'] - rep4_nt_mean) / rep4_nt_std
#FF_df['z_avg_log2'] = (FF_df['avg_log2'] - nt_log2fc_avg_mean) / nt_log2fc_avg_sd

FF_df['z_log2_repavg'] = (FF_df['z_log2_rep1'] + FF_df['z_log2_rep2'] + FF_df['z_log2_rep3'] + FF_df['z_log2_rep4']) / 4

display(FF_df)

### Write dfs to output: one with all guides, one with only targeting guides, and a qBed of targeting guides
FF_df.to_csv('Treg_processed_output_all_counts_df.tsv', sep = '\t', index=False)
FF_df.loc[FF_df['locus']!='nt'].to_csv('Treg_processed_output_targeting_counts_df_noheader.tsv', sep='\t',index=False,header=False)
targeting_df_header_names = FF_df.loc[FF_df['locus']!='nt'].columns.values.tolist()
print(targeting_df_header_names)
FF_df.loc[FF_df['locus']!='nt'].to_csv('Treg_processed_output_targeting.qBed', sep='\t',columns =['chr','start','end','z_log2_repavg','guideID'], index=False,header=False)


## Plot correlation of replicate average z scores
## Note: The following code works 
f, ax = plt.subplots(figsize=(6, 6))
ax.scatter(x=FF_df['z_log2_rep1'], y=FF_df['z_log2_rep2'])
ax.set_xlabel('Z-score Rep1')
ax.set_ylabel('Z-score Rep2')
plt.savefig('Treg_CRISPRi_ZScores_Replicates_eq31_rep1_rep2.pdf')
plt.close("all")

,chr,start,end,strand,locus,guideID,sequence,TG2-GITR-Hi-rep1,TG2-GITR-Hi-rep2,TG2-GITR-Hi-rep3,...,TG2-GITR-Lo-rep4,log2_rep1,log2_rep2,log2_rep3,log2_rep4,z_log2_rep1,z_log2_rep2,z_log2_rep3,z_log2_rep4,z_log2_repavg
0,chr4,156026280,156026302,+,peak_10,66,GGCCTGAAGCCCAGTCTGAG,71,78,86,...,840,-1.547858,-1.550082,-1.805907,-1.507794,-9.059904,-10.703437,-12.348986,-12.597702,-11.177507
1,chr4,156026211,156026233,+,peak_10,67,TCTGCTCTACACTTCACAGA,35,131,149,...,1012,-1.831027,-1.522680,-1.371811,-1.604933,-10.576244,-10.532549,-9.526446,-13.362819,-10.999515
2,chr4,156026240,156026262,+,peak_10,73,CAAACACCTCAGATGTCTGC,240,428,456,...,267,-0.149223,0.456288,0.128660,0.096201,-1.570357,1.808884,0.229769,0.036200,0.126124
3,chr4,156026323,156026345,+,peak_10,48,GAGACAAGGCAAGTTGGAGC,61,110,137,...,1178,-2.025251,-1.977268,-1.939951,-1.716664,-11.616293,-13.367497,-13.220549,-14.242870,-13.111802
4,chr4,156026183,156026205,+,peak_10,74,GTTAGGAGATGTCCAGAAAG,105,226,130,...,1378,-1.702979,-1.573000,-1.648950,-1.784910,-9.890558,-10.846358,-11.328431,-14.780408,-11.711439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439,nontargeting,1,20,.,nt,36,TCGCCCCCCACTACCAAGAA,497,637,631,...,569,0.428610,0.312311,0.109253,0.075891,1.523881,0.910998,0.103583,-0.123773,0.603672
440,nontargeting,1,20,.,nt,37,TCCTGACCTATCCGAAAAAA,375,429,412,...,259,0.037720,0.068372,0.024111,0.230001,-0.569298,-0.610277,-0.450017,1.090080,-0.134878
441,nontargeting,1,20,.,nt,38,CCCACCCCGCTGTTTGCACG,452,796,687,...,586,0.031587,0.385796,0.067637,-0.037161,-0.602138,1.369273,-0.167011,-1.014226,-0.103526
442,nontargeting,1,20,.,nt,39,GGAACCTCGCTAGTACCATT,393,307,407,...,255,0.435624,-0.153382,0.196423,0.281134,1.561437,-1.993201,0.670374,1.492825,0.432859


['chr', 'start', 'end', 'strand', 'locus', 'guideID', 'sequence', 'TG2-GITR-Hi-rep1', 'TG2-GITR-Hi-rep2', 'TG2-GITR-Hi-rep3', 'TG2-GITR-Hi-rep4', 'TG2-GITR-Lo-rep1', 'TG2-GITR-Lo-rep2', 'TG2-GITR-Lo-rep3', 'TG2-GITR-Lo-rep4', 'log2_rep1', 'log2_rep2', 'log2_rep3', 'log2_rep4', 'z_log2_rep1', 'z_log2_rep2', 'z_log2_rep3', 'z_log2_rep4', 'z_log2_repavg']


In [65]:
##### Epifeature peak intersections
"""
Note, there is no EQ file. I need to establish cCREs de novo, which I will do using the ATAC peaks
Find all ATAC peaks that intersect a gRNA, without slop
Track which ATAC peaks intersect the gene of interest (GITR), and exclude. Keep any other TSS-intersecting peak.
"""
### Process ATAC
"""
Pre-process ATAC files
Cat the two reps together
Sort
Merge
(base) davidy@Davids-MacBook-Pro 230724_treg_analysis % cat KS133.Treg.72hrs.ASTARR.insert.rep1.masked.dedup.sorted_peaks.narrowPeak KS133.Treg.72hrs.ASTARR.insert.rep2.masked.dedup.sorted_peaks.narrowPeak > KS133.Treg.72hrs.ASTARR.insert.rep1and2.masked.dedup.sorted_peaks.narrowPeak
(base) davidy@Davids-MacBook-Pro 230724_treg_analysis % sort -k1,1 -k2,2n KS133.Treg.72hrs.ASTARR.insert.rep1and2.masked.dedup.sorted_peaks.narrowPeak > sorted_KS133.Treg.72hrs.ASTARR.insert.rep1and2.masked.dedup.sorted_peaks.narrowPeak
(base) davidy@Davids-MacBook-Pro 230724_treg_analysis % bedtools merge -i sorted_KS133.Treg.72hrs.ASTARR.insert.rep1and2.masked.dedup.sorted_peaks.narrowPeak > merged_sorted_KS133.Treg.72hrs.ASTARR.insert.rep1and2.masked.dedup.sorted_peaks.narrowPeak

"""
targeting_file = 'Treg_processed_output_targeting_counts_df_noheader.tsv'
atac_file = 'merged_sorted_KS133.Treg.72hrs.ASTARR.insert.rep1and2.masked.dedup.sorted_peaks.narrowPeak'

### Intersect ATAC with guides 
fopen = open('ATAC_Treg_guides_intersect.bed', 'w')
subprocess.call(['bedtools','intersect','-a',atac_file,'-b',targeting_file, '-wa','-wb'],stdout=fopen)
fopen.close()

# Intersect
ATAC_targeting_df = pd.read_csv('ATAC_Treg_guides_intersect.bed', sep = '\t', header = None, names = ['atac_chr','atac_start','atac_end']+targeting_df_header_names)

# Add ATAC_CC
ATAC_targeting_df['atac_cc'] = ATAC_targeting_df['atac_chr'].astype(str) + '_' + ATAC_targeting_df['atac_start'].astype(str) + '_' + ATAC_targeting_df['atac_end'].astype(str)

# Add average z_log2_repavg across targeting guides for each atac_cc
ATAC_targeting_df['atac_cc_avg_z_log2'] = ATAC_targeting_df.groupby(['atac_cc'])['z_log2_repavg'].transform('mean')

display(ATAC_targeting_df[['atac_cc','atac_cc_avg_z_log2']].drop_duplicates())
display(ATAC_targeting_df)


"""
Only three peaks have significant effects:
1) TNFRSF18 gene TSS (chr4_156025979_156026724)
2) chr4_156021490_156022916 is an intronic enhancer with great ATAC/H3K27ac. 
Interestingly, the peak of gRNA effects are at the H3K27ac trough, not the ATAC center
Adding in DNase files, the peak of gRNA effects is indeed at the DNase center!
3) chr4_156030602_156033837 is one peak, but has three strong summits
Each summit has a local peak of gRNA activity
Aggregating seems like it would dilute signal. Instead, create a separate, final meta-plot for each enhancer.

"""

### DNase
dhs_file='ENCFF550NKM.bed'
fopen = open('DHS_Treg_guides_intersect.bed', 'w')
subprocess.call(['bedtools','intersect','-a',dhs_file,'-b',targeting_file, '-wa','-wb'],stdout=fopen)
fopen.close()

# Intersect
dhs_targeting_df = pd.read_csv('DHS_Treg_guides_intersect.bed', sep = '\t', header = None, names = ['dhs_chr','dhs_start','dhs_end','i1','i2','i3','i4','i5','i6','i7']+targeting_df_header_names)

# Add dhs_cc
dhs_targeting_df['dhs_cc'] = dhs_targeting_df['dhs_chr'].astype(str) + '_' + dhs_targeting_df['dhs_start'].astype(str) + '_' + dhs_targeting_df['dhs_end'].astype(str)

# Add average z_log2_repavg across targeting guides for each dhs_cc
dhs_targeting_df['dhs_cc_avg_z_log2'] = dhs_targeting_df.groupby(['dhs_cc'])['z_log2_repavg'].transform('mean')

display(dhs_targeting_df[['dhs_cc','dhs_cc_avg_z_log2']].drop_duplicates())
display(dhs_targeting_df)

### Normalize gRNA effects to the dhs_cc_avg_z_log2
dhs_targeting_df['mean_norm_avg_z_log2'] = dhs_targeting_df['z_log2_repavg']/ dhs_targeting_df['dhs_cc_avg_z_log2']
# Write a new mean_norm qBed for all targeting guides
dhs_targeting_df.to_csv('mean_norm_avg_z_log2_targeting_guides.qBed', sep = '\t', columns=['chr','start','end','mean_norm_avg_z_log2','strand','guideID'],index=False,header=False)


## Get just the DHSs with mean effects < -1
strong_effect_dhs_df = dhs_targeting_df.loc[dhs_targeting_df['dhs_cc_avg_z_log2']<= -1][['dhs_chr','dhs_start','dhs_end','dhs_cc']].drop_duplicates()
# Remove the GITR promoter DHS
strong_effect_dhs_df = strong_effect_dhs_df.loc[strong_effect_dhs_df['dhs_cc']!='chr4_156026180_156026380']
display(strong_effect_dhs_df)
# Write out for window intersection
strong_effect_dhs_df.to_csv('strong_effect_nonGITR_dhs.bed', sep = '\t', index=False, header=False)
# Write out an even more minimal
strong_effect_dhs_df.to_csv('strong_effect_nonGITR_dhs_minimal.bed', sep = '\t', columns=['dhs_chr','dhs_start','dhs_end'], index=False, header=False)



### Bedtools window with strong effect dhs
# Using a window of 300 since some summits are fairly close to one another 
# Note that this is different than the K562 analysis, which used 1000 
fopen = open('grna_window500_strong_effect_nonGITR_dhs.bed', 'w')
#subprocess.call(['bedtools','window','-w','300','-a','strong_effect_nonGITR_dhs.bed','-b',targeting_file],stdout=fopen)
subprocess.call(['bedtools','window','-w','1000','-a','strong_effect_nonGITR_dhs.bed','-b','mean_norm_avg_z_log2_targeting_guides.qBed'],stdout=fopen)
fopen.close()




,atac_cc,atac_cc_avg_z_log2
0,chr4_155991689_155993794,0.295815
69,chr4_156004155_156004462,0.110302
75,chr4_156007812_156008869,-0.552416
101,chr4_156010246_156010696,0.115904
110,chr4_156013424_156014943,0.456635
151,chr4_156015521_156016798,0.108025
189,chr4_156021490_156022916,-2.000217
232,chr4_156025979_156026724,-4.755094
257,chr4_156030602_156033837,-1.570431


,atac_chr,atac_start,atac_end,chr,start,end,strand,locus,guideID,sequence,...,log2_rep2,log2_rep3,log2_rep4,z_log2_rep1,z_log2_rep2,z_log2_rep3,z_log2_rep4,z_log2_repavg,atac_cc,atac_cc_avg_z_log2
0,chr4,155991689,155993794,chr4,155991687,155991709,-,peak_1,281,GCCTTTCCTATGTCTATGAC,...,0.326339,0.029425,0.145506,1.179559,0.998484,-0.415468,0.424549,0.546781,chr4_155991689_155993794,0.295815
1,chr4,155991689,155993794,chr4,155992002,155992024,+,peak_1,93,GTAGTAGTCGCAGAGTTGCC,...,0.427560,0.018996,-0.030809,-0.651821,1.629724,-0.483275,-0.964201,-0.117393,chr4_155991689_155993794,0.295815
2,chr4,155991689,155993794,chr4,155991979,155992001,-,peak_1,7,CTGCCCTACGCGTTGGGCGG,...,-0.003629,0.102548,0.081393,-1.649829,-1.059294,0.059985,-0.080439,-0.682394,chr4_155991689_155993794,0.295815
3,chr4,155991689,155993794,chr4,155992020,155992042,-,peak_1,170,GTCGCTGGCGAGAAGCAGCC,...,0.056761,0.140904,0.269617,1.163899,-0.682687,0.309382,1.402116,0.548178,chr4_155991689_155993794,0.295815
4,chr4,155991689,155993794,chr4,155991767,155991789,-,peak_1,314,GCAAAGCCCAGAGGACATGT,...,0.019718,0.232184,0.398698,0.067287,-0.913699,0.902892,2.418825,0.618826,chr4_155991689_155993794,0.295815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,chr4,156030602,156033837,chr4,156033805,156033827,+,peak_11,284,GCTCCAATACAAAACAGGGA,...,-0.028188,0.122441,0.071492,-0.775946,-1.212456,0.189331,-0.158424,-0.489374,chr4_156030602_156033837,-1.570431
338,chr4,156030602,156033837,chr4,156032677,156032699,-,peak_11,136,CCATTAGGAATACAACAATA,...,-0.103451,0.141640,0.045833,-1.182333,-1.681817,0.314164,-0.360523,-0.727627,chr4_156030602_156033837,-1.570431
339,chr4,156030602,156033837,chr4,156032677,156032699,+,peak_11,258,CCATATTGTTGTATTCCTAA,...,0.436109,0.369703,0.146595,-0.798062,1.683038,1.797058,0.433125,0.778790,chr4_156030602_156033837,-1.570431
340,chr4,156030602,156033837,chr4,156031566,156031588,-,peak_11,250,AAACATGGCATGTAGGTACA,...,0.163434,-0.043427,0.032543,0.128791,-0.017440,-0.889157,-0.465203,-0.310752,chr4_156030602_156033837,-1.570431


,dhs_cc,dhs_cc_avg_z_log2
0,chr4_155992320_155992520,0.456007
9,chr4_155992760_155992960,0.092806
11,chr4_155992780_155993240,0.116434
26,chr4_156004200_156004440,0.110302
32,chr4_156008420_156008700,-1.319333
36,chr4_156008420_156008920,-0.617773
48,chr4_156012709_156012967,-0.433370
52,chr4_156013700_156013920,0.646114
57,chr4_156021440_156021680,-0.419953
64,chr4_156022220_156022460,-2.851717


,dhs_chr,dhs_start,dhs_end,i1,i2,i3,i4,i5,i6,i7,...,log2_rep2,log2_rep3,log2_rep4,z_log2_rep1,z_log2_rep2,z_log2_rep3,z_log2_rep4,z_log2_repavg,dhs_cc,dhs_cc_avg_z_log2
0,chr4,155992320,155992520,.,0,.,16.08810,-1,-1,75,...,0.660244,0.177626,0.128381,2.137353,3.080815,0.548148,0.289662,1.513995,chr4_155992320_155992520,0.456007
1,chr4,155992320,155992520,.,0,.,16.08810,-1,-1,75,...,0.147100,0.375245,0.330168,-1.381009,-0.119304,1.833090,1.879040,0.552954,chr4_155992320_155992520,0.456007
2,chr4,155992320,155992520,.,0,.,16.08810,-1,-1,75,...,0.307712,0.317087,-0.076073,-0.032562,0.882318,1.454944,-1.320722,0.245994,chr4_155992320_155992520,0.456007
3,chr4,155992320,155992520,.,0,.,16.08810,-1,-1,75,...,-0.174368,0.321881,0.118739,0.340742,-2.124075,1.486113,0.213716,-0.020876,chr4_155992320_155992520,0.456007
4,chr4,155992320,155992520,.,0,.,16.08810,-1,-1,75,...,0.100846,0.091785,0.273600,-0.053748,-0.407756,-0.009994,1.433488,0.240497,chr4_155992320_155992520,0.456007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,chr4,156033600,156033780,.,0,.,8.98990,-1,-1,75,...,-0.608176,-1.042383,-0.906606,-5.819690,-4.829427,-7.384474,-7.862434,-6.474006,chr4_156033600_156033780,-2.839355
108,chr4,156033600,156033780,.,0,.,8.98990,-1,-1,75,...,0.105834,-0.031836,-0.212958,-0.778251,-0.376651,-0.813795,-2.398896,-1.091898,chr4_156033600_156033780,-2.839355
109,chr4,156033600,156033780,.,0,.,8.98990,-1,-1,75,...,-0.241044,-0.422412,-0.521194,-2.624139,-2.539889,-3.353358,-4.826725,-3.336028,chr4_156033600_156033780,-2.839355
110,chr4,156033600,156033780,.,0,.,8.98990,-1,-1,75,...,-0.238502,-0.174071,-0.303167,-1.592547,-2.524032,-1.738621,-3.109432,-2.241158,chr4_156033600_156033780,-2.839355


,dhs_chr,dhs_start,dhs_end,dhs_cc
32,chr4,156008420,156008700,chr4_156008420_156008700
64,chr4,156022220,156022460,chr4_156022220_156022460
80,chr4,156030680,156031020,chr4_156030680_156031020
91,chr4,156031880,156032260,chr4_156031880_156032260
99,chr4,156033020,156033220,chr4_156033020_156033220
104,chr4,156033600,156033780,chr4_156033600_156033780


In [67]:
##### Further process dhs_grna_df for deepTools

#dhs_grna_df = pd.read_csv('grna_window500_strong_effect_nonGITR_dhs.bed', sep = '\t', header=None, names = ['dhs_chr','dhs_start','dhs_end','dhs_cc']+targeting_df_header_names, usecols=['chr','start','end','strand','guideID','z_log2_repavg','dhs_cc'])
dhs_grna_df = pd.read_csv('grna_window500_strong_effect_nonGITR_dhs.bed', sep = '\t', header=None, names = ['dhs_chr','dhs_start','dhs_end','dhs_cc']+['chr','start','end','z_log2_repavg','strand','guideID']) # Note z_log2_repavg is actually the dhs mean_norm_z_log2_repavg
display(dhs_grna_df)

lod = []

#krab_window=0
krab_window = 150

for i, row in dhs_grna_df.iterrows():
    if row['strand']=='+':
        for i in range(row['end']-3-krab_window,row['end']+krab_window,1):
            adf = {'chr':row['chr'],
                  'start':i,
                  'end':i+1,
                  'effect':row['z_log2_repavg']}
            lod.append(adf)
    else:
        for i in range(row['start']-krab_window,row['start']+3+krab_window,1):
            adf = {'chr':row['chr'],
                  'start':i,
                  'end':i+1,
                  'effect':row['z_log2_repavg']}
            lod.append(adf)


expanded_df = pd.DataFrame(lod)
expanded_df.to_csv('expanded_dhs_grna.bedGraph', sep = '\t', index = False, header = False)

fopen = open('sorted_expanded_dhs_grna.bedGraph', 'w')
subprocess.call(['sort','-k1,1','-k2,2n', 'expanded_dhs_grna.bedGraph'],stdout=fopen)
fopen.close()

# Merge, the -d -1 flag is very important to prevent book-ended features from being overlapped, keeping resolution of averaging at per-base. 
fopen = open('averaged_sorted_expanded_dhs_grna.bedGraph', 'w')
subprocess.call(['bedtools','merge','-c','4','-o','mean','-d','-1' ,'-i','sorted_expanded_dhs_grna.bedGraph'],stdout=fopen)
fopen.close()

# Finally run bedGraphtoBigWig
#fopen = open('averaged_expanded_max.bw','w')
subprocess.call(['/Users/davidy/pythonscripts/bedGraphToBigWig','averaged_sorted_expanded_dhs_grna.bedGraph','/Users/davidy/misc_resources/mm10.chrom.sizes','averaged_sorted_expanded_dhs_grna.bw'])
#fopen.close()

,dhs_chr,dhs_start,dhs_end,dhs_cc,chr,start,end,z_log2_repavg,strand,guideID
0,chr4,156008420,156008700,chr4_156008420_156008700,chr4,156008593,156008615,1.230585,+,88
1,chr4,156008420,156008700,chr4_156008420_156008700,chr4,156008520,156008542,0.736820,-,13
2,chr4,156008420,156008700,chr4_156008420_156008700,chr4,156008651,156008673,1.748869,-,84
3,chr4,156008420,156008700,chr4_156008420_156008700,chr4,156008674,156008696,0.283725,-,33
4,chr4,156008420,156008700,chr4_156008420_156008700,chr4,156008593,156008615,2.628070,+,88
...,...,...,...,...,...,...,...,...,...,...
88,chr4,156033600,156033780,chr4_156033600_156033780,chr4,156033725,156033747,-0.076402,-,109
89,chr4,156033600,156033780,chr4_156033600_156033780,chr4,156033633,156033655,2.280097,-,135
90,chr4,156033600,156033780,chr4_156033600_156033780,chr4,156033702,156033724,0.384559,+,154
91,chr4,156033600,156033780,chr4_156033600_156033780,chr4,156033747,156033769,1.174924,-,115


0

In [68]:
##### deepTools
# Compute matrix for just the DHS and H3K27ac
#subprocess.call(['computeMatrix', 'reference-point', '-R', 'strong_effect_nonGITR_dhs_minimal.bed', '-S', 'KS133.Treg.72hrs.ASTARR.insert.rep1.masked.dedup.sorted.rpkm.bw', 'H3K27acChIPseq_Treg1.masked.dedup.sorted.rpkm.bw', '-o', 'treg_matrix_dhs_h3k27ac.tab.gz', '--referencePoint', 'center', '-b', '300', '-a', '300', '--averageTypeBins', 'mean', '--skipZeros'])
subprocess.call(['computeMatrix', 'reference-point', '-R', 'strong_effect_nonGITR_dhs_minimal.bed', '-S', 'H3K27acChIPseq_Treg1.masked.dedup.sorted.rpkm.bw', '-o', 'treg_matrix_dhs_h3k27ac.tab.gz', '--referencePoint', 'center', '-b', '300', '-a', '300', '--averageTypeBins', 'mean', '--skipZeros'])

# Compute matrix for sgRNAs
outname='treg_matrix_sgrnas_krabwindow' + str(krab_window) + '.tab.gz'
subprocess.call(['computeMatrix', 'reference-point', '-R', 'strong_effect_nonGITR_dhs_minimal.bed', '-S', 'averaged_sorted_expanded_dhs_grna.bw', '-o', outname, '--referencePoint', 'center', '-b', '300', '-a', '300'])


# plotProfiles
subprocess.call(['plotProfile', '-m', 'treg_matrix_dhs_h3k27ac.tab.gz', '-out', 'treg_DHSandH3K27acEnhancer_bw.pdf', '--perGroup', '--plotHeight', '6', '--plotWidth', '10'])
subprocess.call(['plotProfile', '-m', outname, '-out', outname.split('.')[0]+'.pdf', '--perGroup', '--plotHeight', '6', '--plotWidth', '10'])


0

In [78]:
##### Alternative approach:
##### Normalize gRNA effects based on the entire ATAC peak, which is the original element that is tiled
##### 


### Process ATAC
"""
Pre-process ATAC files
Cat the two reps together
Sort
Merge
(base) davidy@Davids-MacBook-Pro 230724_treg_analysis % cat KS133.Treg.72hrs.ASTARR.insert.rep1.masked.dedup.sorted_peaks.narrowPeak KS133.Treg.72hrs.ASTARR.insert.rep2.masked.dedup.sorted_peaks.narrowPeak > KS133.Treg.72hrs.ASTARR.insert.rep1and2.masked.dedup.sorted_peaks.narrowPeak
(base) davidy@Davids-MacBook-Pro 230724_treg_analysis % sort -k1,1 -k2,2n KS133.Treg.72hrs.ASTARR.insert.rep1and2.masked.dedup.sorted_peaks.narrowPeak > sorted_KS133.Treg.72hrs.ASTARR.insert.rep1and2.masked.dedup.sorted_peaks.narrowPeak
(base) davidy@Davids-MacBook-Pro 230724_treg_analysis % bedtools merge -i sorted_KS133.Treg.72hrs.ASTARR.insert.rep1and2.masked.dedup.sorted_peaks.narrowPeak > merged_sorted_KS133.Treg.72hrs.ASTARR.insert.rep1and2.masked.dedup.sorted_peaks.narrowPeak

"""
targeting_file = 'Treg_processed_output_targeting_counts_df_noheader.tsv'
atac_file = 'merged_sorted_KS133.Treg.72hrs.ASTARR.insert.rep1and2.masked.dedup.sorted_peaks.narrowPeak'

### Intersect ATAC with guides 
fopen = open('ATAC_Treg_guides_intersect.bed', 'w')
subprocess.call(['bedtools','intersect','-a',atac_file,'-b',targeting_file, '-wa','-wb'],stdout=fopen)
fopen.close()

# Intersect
ATAC_targeting_df = pd.read_csv('ATAC_Treg_guides_intersect.bed', sep = '\t', header = None, names = ['atac_chr','atac_start','atac_end']+targeting_df_header_names)

# Add ATAC_CC
ATAC_targeting_df['atac_cc'] = ATAC_targeting_df['atac_chr'].astype(str) + '_' + ATAC_targeting_df['atac_start'].astype(str) + '_' + ATAC_targeting_df['atac_end'].astype(str)

# Add average z_log2_repavg across targeting guides for each atac_cc
ATAC_targeting_df['atac_cc_avg_z_log2'] = ATAC_targeting_df.groupby(['atac_cc'])['z_log2_repavg'].transform('mean')

display(ATAC_targeting_df[['atac_cc','atac_cc_avg_z_log2']].drop_duplicates())
display(ATAC_targeting_df)


"""
Only three peaks have significant effects:
1) TNFRSF18 gene TSS (chr4_156025979_156026724)
2) chr4_156021490_156022916 is an intronic enhancer with great ATAC/H3K27ac. 
Interestingly, the peak of gRNA effects are at the H3K27ac trough, not the ATAC center
Adding in DNase files, the peak of gRNA effects is indeed at the DNase center!
3) chr4_156030602_156033837 is one peak, but has three strong summits
Each summit has a local peak of gRNA activity
Aggregating seems like it would dilute signal. Instead, create a separate, final meta-plot for each enhancer.

"""

### Normalize gRNA effects to the dhs_cc_avg_z_log2
ATAC_targeting_df['mean_norm_avg_z_log2'] = ATAC_targeting_df['z_log2_repavg']/ ATAC_targeting_df['atac_cc_avg_z_log2']
# Write a new mean_norm qBed for all targeting guides
ATAC_targeting_df.to_csv('atac_mean_norm_avg_z_log2_targeting_guides.qBed', sep = '\t', columns=['chr','start','end','mean_norm_avg_z_log2','strand','guideID'],index=False,header=False)


## Get just the DHSs with mean effects < -1
strong_effect_atac_df = ATAC_targeting_df.loc[ATAC_targeting_df['atac_cc_avg_z_log2']<= -1][['atac_chr','atac_start','atac_end','atac_cc']].drop_duplicates()
# Remove the GITR promoter DHS
strong_effect_atac_df = strong_effect_atac_df.loc[strong_effect_atac_df['atac_cc']!='chr4_156025979_156026724']
display(strong_effect_atac_df)
# Write out for window intersection
strong_effect_atac_df.to_csv('strong_effect_nonGITR_atac.bed', sep = '\t', index=False, header=False)
# Write out an even more minimal
strong_effect_atac_df.to_csv('strong_effect_nonGITR_atac_minimal.bed', sep = '\t', columns=['atac_chr','atac_start','atac_end'], index=False, header=False)



### Bedtools window with strong effect dhs
# Using a window of 300 since some summits are fairly close to one another 
# Note that this is different than the K562 analysis, which used 1000 
fopen = open('grna_window500_strong_effect_nonGITR_atac.bed', 'w')
#subprocess.call(['bedtools','window','-w','300','-a','strong_effect_nonGITR_atac.bed','-b',targeting_file],stdout=fopen)
subprocess.call(['bedtools','window','-w','1000','-a','strong_effect_nonGITR_atac.bed','-b','atac_mean_norm_avg_z_log2_targeting_guides.qBed'],stdout=fopen)
fopen.close()



##### Further process dhs_grna_df for deepTools

atac_grna_df = pd.read_csv('grna_window500_strong_effect_nonGITR_atac.bed', sep = '\t', header=None, names = ['atac_chr','atac_start','atac_end','atac_cc']+['chr','start','end','z_log2_repavg','strand','guideID']) # Note z_log2_repavg is actually the dhs mean_norm_z_log2_repavg
display(atac_grna_df)

lod = []

#krab_window=0
krab_window = 150

for i, row in atac_grna_df.iterrows():
    if row['strand']=='+':
        for i in range(row['end']-3-krab_window,row['end']+krab_window,1):
            adf = {'chr':row['chr'],
                  'start':i,
                  'end':i+1,
                  'effect':row['z_log2_repavg']}
            lod.append(adf)
    else:
        for i in range(row['start']-krab_window,row['start']+3+krab_window,1):
            adf = {'chr':row['chr'],
                  'start':i,
                  'end':i+1,
                  'effect':row['z_log2_repavg']}
            lod.append(adf)


expanded_df = pd.DataFrame(lod)
expanded_df.to_csv('expanded_atac_grna.bedGraph', sep = '\t', index = False, header = False)

fopen = open('sorted_expanded_atac_grna.bedGraph', 'w')
subprocess.call(['sort','-k1,1','-k2,2n', 'expanded_atac_grna.bedGraph'],stdout=fopen)
fopen.close()

# Merge, the -d -1 flag is very important to prevent book-ended features from being overlapped, keeping resolution of averaging at per-base. 
fopen = open('averaged_sorted_expanded_atac_grna.bedGraph', 'w')
subprocess.call(['bedtools','merge','-c','4','-o','mean','-d','-1' ,'-i','sorted_expanded_atac_grna.bedGraph'],stdout=fopen)
fopen.close()

# Finally run bedGraphtoBigWig
#fopen = open('averaged_expanded_max.bw','w')
subprocess.call(['/Users/davidy/pythonscripts/bedGraphToBigWig','averaged_sorted_expanded_atac_grna.bedGraph','/Users/davidy/misc_resources/mm10.chrom.sizes','averaged_sorted_expanded_atac_grna.bw'])
#fopen.close()



##### deepTools
"""
Here, we will use the DHS-based elements as the reference anchors!!!

"""
deeptools_window = '500'
# Compute matrix for just the DHS and H3K27ac
#subprocess.call(['computeMatrix', 'reference-point', '-R', 'strong_effect_nonGITR_dhs_minimal.bed', '-S', 'KS133.Treg.72hrs.ASTARR.insert.rep1.masked.dedup.sorted.rpkm.bw', 'H3K27acChIPseq_Treg1.masked.dedup.sorted.rpkm.bw', '-o', 'treg_matrix_dhs_h3k27ac.tab.gz', '--referencePoint', 'center', '-b', '300', '-a', '300', '--averageTypeBins', 'mean', '--skipZeros'])
subprocess.call(['computeMatrix', 'reference-point', '-R', 'strong_effect_nonGITR_dhs_minimal.bed', '-S', 'H3K27acChIPseq_Treg1.masked.dedup.sorted.rpkm.bw', '-o', 'treg_matrix_h3k27ac.tab.gz', '--referencePoint', 'center', '-b', deeptools_window, '-a', deeptools_window, '--averageTypeBins', 'mean', '--skipZeros'])

subprocess.call(['computeMatrix', 'reference-point', '-R', 'strong_effect_nonGITR_dhs_minimal.bed', '-S', 'ENCFF451FMO.bigWig', '-o', 'treg_matrix_dhs.tab.gz', '--referencePoint', 'center', '-b', deeptools_window, '-a', deeptools_window, '--averageTypeBins', 'mean', '--skipZeros'])

# Compute matrix for sgRNAs
outname='treg_matrix_sgrnas_krabwindow' + str(krab_window) + '_atacNorm.tab.gz'
subprocess.call(['computeMatrix', 'reference-point', '-R', 'strong_effect_nonGITR_dhs_minimal.bed', '-S', 'averaged_sorted_expanded_atac_grna.bw', '-o', outname, '--referencePoint', 'center', '-b', deeptools_window, '-a', deeptools_window])


# plotProfiles
subprocess.call(['plotProfile', '-m', 'treg_matrix_h3k27ac.tab.gz', '-out', 'treg_H3K27ac_bw.pdf', '--perGroup', '--plotHeight', '6', '--plotWidth', '10'])
subprocess.call(['plotProfile', '-m', 'treg_matrix_dhs.tab.gz', '-out', 'treg_DHS_bw.pdf', '--perGroup', '--plotHeight', '6', '--plotWidth', '10'])
subprocess.call(['plotProfile', '-m', outname, '-out', outname.split('.')[0]+'.pdf', '--perGroup', '--plotHeight', '6', '--plotWidth', '10'])


,atac_cc,atac_cc_avg_z_log2
0,chr4_155991689_155993794,0.295815
69,chr4_156004155_156004462,0.110302
75,chr4_156007812_156008869,-0.552416
101,chr4_156010246_156010696,0.115904
110,chr4_156013424_156014943,0.456635
151,chr4_156015521_156016798,0.108025
189,chr4_156021490_156022916,-2.000217
232,chr4_156025979_156026724,-4.755094
257,chr4_156030602_156033837,-1.570431


,atac_chr,atac_start,atac_end,chr,start,end,strand,locus,guideID,sequence,...,log2_rep2,log2_rep3,log2_rep4,z_log2_rep1,z_log2_rep2,z_log2_rep3,z_log2_rep4,z_log2_repavg,atac_cc,atac_cc_avg_z_log2
0,chr4,155991689,155993794,chr4,155991687,155991709,-,peak_1,281,GCCTTTCCTATGTCTATGAC,...,0.326339,0.029425,0.145506,1.179559,0.998484,-0.415468,0.424549,0.546781,chr4_155991689_155993794,0.295815
1,chr4,155991689,155993794,chr4,155992002,155992024,+,peak_1,93,GTAGTAGTCGCAGAGTTGCC,...,0.427560,0.018996,-0.030809,-0.651821,1.629724,-0.483275,-0.964201,-0.117393,chr4_155991689_155993794,0.295815
2,chr4,155991689,155993794,chr4,155991979,155992001,-,peak_1,7,CTGCCCTACGCGTTGGGCGG,...,-0.003629,0.102548,0.081393,-1.649829,-1.059294,0.059985,-0.080439,-0.682394,chr4_155991689_155993794,0.295815
3,chr4,155991689,155993794,chr4,155992020,155992042,-,peak_1,170,GTCGCTGGCGAGAAGCAGCC,...,0.056761,0.140904,0.269617,1.163899,-0.682687,0.309382,1.402116,0.548178,chr4_155991689_155993794,0.295815
4,chr4,155991689,155993794,chr4,155991767,155991789,-,peak_1,314,GCAAAGCCCAGAGGACATGT,...,0.019718,0.232184,0.398698,0.067287,-0.913699,0.902892,2.418825,0.618826,chr4_155991689_155993794,0.295815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,chr4,156030602,156033837,chr4,156033805,156033827,+,peak_11,284,GCTCCAATACAAAACAGGGA,...,-0.028188,0.122441,0.071492,-0.775946,-1.212456,0.189331,-0.158424,-0.489374,chr4_156030602_156033837,-1.570431
338,chr4,156030602,156033837,chr4,156032677,156032699,-,peak_11,136,CCATTAGGAATACAACAATA,...,-0.103451,0.141640,0.045833,-1.182333,-1.681817,0.314164,-0.360523,-0.727627,chr4_156030602_156033837,-1.570431
339,chr4,156030602,156033837,chr4,156032677,156032699,+,peak_11,258,CCATATTGTTGTATTCCTAA,...,0.436109,0.369703,0.146595,-0.798062,1.683038,1.797058,0.433125,0.778790,chr4_156030602_156033837,-1.570431
340,chr4,156030602,156033837,chr4,156031566,156031588,-,peak_11,250,AAACATGGCATGTAGGTACA,...,0.163434,-0.043427,0.032543,0.128791,-0.017440,-0.889157,-0.465203,-0.310752,chr4_156030602_156033837,-1.570431


,atac_chr,atac_start,atac_end,atac_cc
189,chr4,156021490,156022916,chr4_156021490_156022916
257,chr4,156030602,156033837,chr4_156030602_156033837


,atac_chr,atac_start,atac_end,atac_cc,chr,start,end,z_log2_repavg,strand,guideID
0,chr4,156021490,156022916,chr4_156021490_156022916,chr4,156022280,156022302,1.155394,+,9
1,chr4,156021490,156022916,chr4_156021490_156022916,chr4,156022301,156022323,0.527083,+,91
2,chr4,156021490,156022916,chr4_156021490_156022916,chr4,156022043,156022065,2.213361,-,71
3,chr4,156021490,156022916,chr4_156021490_156022916,chr4,156021993,156022015,0.318656,+,59
4,chr4,156021490,156022916,chr4_156021490_156022916,chr4,156022260,156022282,0.445233,-,128
...,...,...,...,...,...,...,...,...,...,...
123,chr4,156030602,156033837,chr4_156030602_156033837,chr4,156033805,156033827,0.311617,+,284
124,chr4,156030602,156033837,chr4_156030602_156033837,chr4,156032677,156032699,0.463330,-,136
125,chr4,156030602,156033837,chr4_156030602_156033837,chr4,156032677,156032699,-0.495908,+,258
126,chr4,156030602,156033837,chr4_156030602_156033837,chr4,156031566,156031588,0.197877,-,250


0